In [2]:
import os
import PIL.Image
import PIL.ExifTags
import csv
import pandas as pd

In [80]:
path = './Images'


In [7]:
for i in os.listdir(path):
    image_path = os.path.join(path, i)
    img = PIL.Image.open(image_path)
    exif_data = img._getexif()
    print(exif_data)

{34853: {0: b'\x02\x03\x00\x00', 1: 'S', 2: (0.0, 27.658, 0.0), 3: 'W', 4: (80.0, 27.464, 0.0), 5: b'\x00', 6: 6.2, 7: (17.0, 15.0, 40.0), 8: '11', 16: 'T', 17: 347.25, 18: 'WGS-84\x00  ', 29: '2016:06:01'}, 296: 2, 34665: 20418, 271: 'NIKON', 272: 'COOLPIX AW100\x00 ', 305: 'Microsoft Windows Photo Viewer 6.1.7600.16385', 274: 1, 306: '2016:06:17 14:50:50', 531: 2, 282: 300.0, 283: 300.0, 59932: b'\x1c\xea\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [81]:
df = pd.DataFrame()

for i in os.listdir(path):
    image_path = os.path.join(path, i)
    img = PIL.Image.open(image_path)
    
    #Get EXIF data    
    exif = {
        PIL.ExifTags.TAGS[k]: v
        for k, v in img._getexif().items()
        if k in PIL.ExifTags.TAGS
        }
    print(exif)
    
    #Get GPS data
    gps={}
    for k, v in exif['GPSInfo'].items():
        geo_tag = PIL.ExifTags.GPSTAGS.get(k)
        gps[geo_tag]=v
 
    #Get Latitude and Longitude
    lat = gps['GPSLatitude']
    long = gps['GPSLongitude']
    
    #Convert to degrees
    lat = float(lat[0]+(lat[1]/60)+(lat[2]/(3600*100)))
    long = float(long[0]+(long[1]/60)+(long[2]/(3600*100)))
    
    #Negative if LatitudeRef:S or LongitudeRef:W
    if gps['GPSLatitudeRef']=='S':
        lat = -lat
    if gps['GPSLongitudeRef']=='W':
        long = -long
        
    #Get DateTime metadata
    
    #print(lat, long)
    coords = {'latitude': lat, 'longitude': long}
    
    #print(img.getxmp().items())
    
    df_loc = pd.DataFrame(coords, index=[0])
    df = pd.concat([df, df_loc], ignore_index=True)
    

{'GPSInfo': {0: b'\x02\x03\x00\x00', 1: 'N', 2: (27.0, 44.1915, 0.0), 3: 'E', 4: (85.0, 18.1454, 0.0), 5: b'\x00', 6: 1276.5, 7: (5.0, 38.0, 16.835), 8: '22', 9: 'A', 10: '3', 11: 2.7, 16: 'M', 17: 116.0, 18: 'WGS-84', 29: '2015:06:15'}, 'ResolutionUnit': 2, 'ExifOffset': 360, 'Make': 'Canon', 'Model': 'Canon EOS 7D Mark II', 'YResolution': 72.0, 'Orientation': 1, 'DateTime': '2015:06:15 12:26:55', 'YCbCrPositioning': 2, 'Copyright': '', 'XResolution': 72.0, 'Artist': '', 'ExifVersion': b'0230', 'ComponentsConfiguration': b'\x01\x02\x03\x00', 'ShutterSpeedValue': 7.375, 'DateTimeOriginal': '2015:06:15 12:26:55', 'DateTimeDigitized': '2015:06:15 12:26:55', 'ApertureValue': 6.0, 'ExposureBiasValue': -0.3333333333333333, 'MeteringMode': 5, 'UserComment': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [82]:
df.head()

,latitude,longitude
0,2.773653e+01,8.530242e+01
1,2.773336e+01,8.530002e+01
2,2.773535e+01,8.530850e+01
3,2.773646e+01,8.530309e+01
4,1.705684e+07,1.705684e+07


In [15]:
df.to_csv('exif_data.csv', index=False)